In [ ]:
# default_exp models.utils

In [ ]:
# export
import torch
from torch import nn
import torch.nn.functional as F
from functools import partial
from enum import Enum
from fastcore.all import delegates, L

## Basic Utilities -

In [ ]:
# export
norm_types = (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d,
              nn.InstanceNorm1d, nn.InstanceNorm2d, nn.InstanceNorm3d, nn.LayerNorm,)

In [ ]:
# export
def requires_grad(m):
    "Check if the first parameter of `m` requires grad or not"
    ps = list(m.parameters())
    return ps[0].requires_grad if len(ps) > 0 else False


def init_default(m, func=nn.init.kaiming_normal_):
    "Initialize `m` weights with `func` and set `bias` to 0."
    if func:
        if hasattr(m, "weight"):
            func(m.weight)
        if hasattr(m, "bias") and hasattr(m.bias, "data"):
            m.bias.data.fill_(0.0)
    return m


def cond_init(m, func):
    "Apply `init_default` to `m` unless it's a batchnorm module"
    if (not isinstance(m, norm_types)) and requires_grad(m):
        init_default(m, func)


def apply_leaf(m, f):
    "Apply `f` to children of `m`."
    c = m.children()
    if isinstance(m, nn.Module):
        f(m)
    for l in c:
        apply_leaf(l, f)


def apply_init(m, func=nn.init.kaiming_normal_):
    "Initialize all non-batchnorm layers of `m` with `func`."
    apply_leaf(m, partial(cond_init, func=func))

In [ ]:
# export
def cut_model(model: nn.Module, n: int = -2):
    "cuts `model` layers upto `n`"
    ls = list(model.children())[:n]
    encoder = nn.Sequential(*ls)
    return encoder

In [ ]:
# export
@torch.no_grad()
def num_features_model(m, ch_int: int = 3):
    "Return the number of output features for `m`."
    sz = 32
    while True:
        try:
            x = torch.zeros((32, ch_int, sz, sz))
            dummy_out = m.eval()(x)
            return dummy_out.shape[1]
        except Exception as e:
            sz *= 2
            if sz > 2048: raise e

In [ ]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted 01_data.datasets.ipynb.
Converted 01a_data.mixmethods.ipynb.
Converted 02_losses.ipynb.
Converted 03_models.utils.ipynb.
No export destination, ignored:
# export
import torch
from torch import nn
import torch.nn.functional as F
No export destination, ignored:
# export
# mish activation from : https://github.com/fastai/fastai/blob/master/fastai/layers.py#L549
@torch.jit.script
def _mish_jit_fwd(x):
    return x.mul(torch.tanh(F.softplus(x)))


@torch.jit.script
def _mish_jit_bwd(x, grad_output):
    x_sigmoid = torch.sigmoid(x)
    x_tanh_sp = F.softplus(x).tanh()
    return grad_output.mul(x_tanh_sp + x * x_sigmoid * (1 - x_tanh_sp * x_tanh_sp))


class MishJitAutoFn(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return _mish_jit_fwd(x)

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        return _mish_jit_bwd(x, grad_output)


def mish(x):
    r

AttributeError: 'NoneType' object has no attribute 'start'